# Parallelization Workflow

In this notebook, we'll explore parallelization as a powerful workflow pattern for building AI agents.
Using Kotlin's coroutines and Claude via LangChain4j,
we'll implement examples showing how to run LLM tasks concurrently for improved performance and reliability.

## What is parallelization?

Parallelization involves running multiple LLM tasks simultaneously and then combining their results.
This approach offers two main variations:

![Parallelization Workflow Diagram](image/parallelization.svg)

### When to use parallelization
- Sectioning — Breaking a complex task into independent subtasks that can be processed in parallel.
  Each LLM instance focuses on a specific aspect of the problem, allowing for more focused attention and often better results.
- Voting — Running the same task multiple times with different prompts or configurations to gather diverse perspectives,
  then aggregating the results for a more reliable answer.

## Setting up environment

Let's start by configuring the Kotlin notebook with the necessary dependencies:

In [1]:
%useLatestDescriptors
%use langchain4j(model = anthropic)
%use coroutines

Need an API key for accessing Claude:

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY")

## Creating LLM interface

Next, define a helper function for making LLM calls

In [3]:
import dev.langchain4j.data.message.SystemMessage
import dev.langchain4j.data.message.UserMessage

suspend fun llmCall(
    prompt: String,
    systemPrompt: String? = null,
    model: AnthropicChatModelName = AnthropicChatModelName.CLAUDE_3_7_SONNET_20250219
): String {
    val client = AnthropicChatModel.builder()
        .apiKey(apiKey)
        .modelName(model)
        .maxTokens(4096)
        .temperature(0.1)
        .build()

    return withContext(Dispatchers.IO) {
        val messages = buildList {
            systemPrompt?.let { add(SystemMessage(it)) }
            add(UserMessage(prompt))
        }
        val response = client.chat(messages)
        response.aiMessage().text()
    }
}

## Implementing parallel processing

Now implement the core function for parallel processing:

In [4]:
/**
 * Process multiple inputs concurrently with the same prompt.
 */
suspend fun parallel(prompt: String, inputs: List<String>, nWorkers: Int = 3): List<String> = coroutineScope {
    // Create a dispatcher with a fixed thread pool
    val dispatcher = Dispatchers.IO.limitedParallelism(nWorkers)

    // Launch a coroutine for each input and collect the results
    inputs.map { input: String ->
        async(dispatcher) { llmCall("$prompt\nInput: $input") }
    }.awaitAll()
}

## Example with stakeholder impact analysis

Let's test implementation with a business scenario —
analyzing how market changes affect different stakeholder groups:

In [5]:
val stakeholders = listOf(
    """
    Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns
    """,

    """
    Employees:
    - Job security worries
    - Need new skills
    - Want clear direction
    """,

    """
    Investors:
    - Expect growth
    - Want cost control
    - Risk concerns
    """,

    """
    Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions
    """
)

Run parallel processing

In [6]:
runBlocking {
    val impactResults = parallel(
        """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
        stakeholders
    )


    impactResults.forEach {
        print(it)
    }
}

# Market Impact Analysis for Price-Sensitive, Tech-Focused, Environmentally Conscious Customers

## Key Market Changes & Impacts

### HIGH PRIORITY
**1. Rising Inflation & Cost Pressures**
* Customers will become increasingly price-sensitive and may delay purchases
* Value perception will be critical to maintain loyalty
* Potential shift toward lower-cost alternatives or extending product lifecycles

**2. Technology Acceleration**
* Growing expectation gap between current offerings and desired tech capabilities
* Increased comparison shopping based on technical specifications
* Willingness to switch brands for meaningful technological improvements

**3. Environmental Regulation & Awareness**
* Heightened scrutiny of environmental claims and practices
* Growing demand for transparent sustainability information
* Potential premium acceptance for genuinely eco-friendly options

## Recommended Actions

### IMMEDIATE ACTIONS (0-3 months)
1. **Value Communication Strategy**
   * Develop clea

## How it works

1. Define a common analysis prompt that will be applied to each stakeholder group
2. Launch multiple concurrent LLM tasks, each analyzing a different stakeholder
3. Kotlin's coroutines handle the concurrency, managing thread allocation efficiently
4. Results are collected in the same order as the inputs

This approach significantly improves processing time compared to sequential processing, especially for larger numbers of inputs.

## Conclusion

Parallelization offers a powerful approach to building more effective AI agents by leveraging concurrency.
Whether through sectioning complex tasks or gathering diverse perspectives through voting,
this pattern helps create systems that are faster, more reliable, and produce higher quality results.

Kotlin's coroutines provide an elegant way to implement these patterns,
making concurrent LLM processing both efficient and readable.